In [7]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point
import random

weather_csv = gpd.read_file("./기상청_종관기상관측_ASOS.csv", encoding='cp949')
accident_csv = gpd.read_file("제주특별자치도_수난사고 구조출동 현황_2011_2020.csv")
length = len(accident_csv)

idx = int(random.random() * length)
print("coordinate : " + str(accident_csv['loc_info_x'][idx]) + ", " + str(accident_csv['loc_info_y'][idx]))

coordinate : 161519.56, 53756.12


0       131634.8906
1       157287.3438
2       160248.2969
3       124546.5078
4       124546.5078
           ...     
1542      153240.39
1543      156449.07
1544      149236.92
1545      145783.46
1546      156470.21
Name: loc_info_x, Length: 1547, dtype: object

In [19]:
url = 'https://kauth.kakao.com/oauth/token'
rest_api_key = '359e276820f89feab230a8eb1eeadedd'
redirect_uri = 'http://localhost:8888/' # APP에서 등록한 redirect_url
authorize_code = '-Ti1X-TEIwWEg8lHx3xTPOKZk5TdUQMW4M_joY99Vw_GMUHg78lt7g-McQx5fhPTvvUt0worDR4AAAGCFOaPpA'
 
data = {
        'grant_type':'authorization_code',
        'client_id':rest_api_key,
        'redirect_uri':redirect_uri,
        'code': authorize_code,
    }

response = requests.post(url, data=data)
tokens = response.json()
print(tokens["access_token"])

aV1WHLM4r0WP7mt895Qpb9VErm3x_GCcfwf-_syqCilvVAAAAYIU5yyZ


In [32]:
import requests

method = "GET"
url = "https://dapi.kakao.com/v2/local/geo/transcoord.json"
params = {'x' : str(accident_csv['loc_info_x'][idx]), 
          'y' : str(accident_csv['loc_info_y'][idx]),
          'input_coord' : 'WKTM',
          'output_coord' : 'WGS84'}
header = {'authorization': 'KakaoAK 359e276820f89feab230a8eb1eeadedd'}

response = requests.request(method=method, url=url, headers=header, params=params )

tokens = response.json()

print(response)
print(tokens)


<Response [200]>
{'meta': {'total_count': 1}, 'documents': [{'x': 125.44677692988476, 'y': 33.05006202689936}]}
